# LinkdIn Jobs Scrapper

## Imports

In [1]:
from selenium import webdriver
import time
import pandas as pd
import os

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import re
from bs4 import BeautifulSoup

import numpy as np

In [2]:
linkdInJobsUrl = "https://www.linkedin.com/jobs/search?keywords=Microsoft%20Power%20Platform&location=India&geoId=102713980&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"

## WebDriver

In [3]:
#Creating driver instance
driver1 = webdriver.Chrome(executable_path=r'".\\chromedriver.exe"')

#fetching the data
driver1.get(linkdInJobsUrl)
driver1.implicitly_wait(15)

C:\Users\vibhu\AppData\Local\Temp\ipykernel_31348\84967656.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(executable_path=r'".\\chromedriver.exe"')
Incompatible release of chromedriver (version 114.0.5735.16) detected in PATH: D:\Coding\projectsForResume\PowerBI\linkdIn_analyzer\linkdin_scrapper\chromedriver.exe


### Finding the total results

In [4]:
jobResultCountObj = driver1.find_element(By.CLASS_NAME,'results-context-header__job-count')
jobResultCountHtml = jobResultCountObj.get_attribute("outerHTML")

In [5]:
jobResultCountHtml

'<span class="results-context-header__job-count">1,000+</span>'

In [6]:
soup1 = BeautifulSoup(jobResultCountHtml,"html.parser")

In [7]:
soup1.text

'1,000+'

In [8]:
jobResultCountText = ""
for i in soup1.text:
    if i >= '0' and i <= '9' :
        jobResultCountText += i

jobResultCount = pd.to_numeric(jobResultCountText)

In [9]:
jobResultCount

1000

### Calculating the loops needed
#### Assumption : 20 items are being loaded in single scroll

In [10]:
loops = 200

## Making sure that driver keeps scrolling to the bottom of the page and loads new stuff

In [ ]:
i=1
while i<=loops:
    #Scroll to bottom
    driver1.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i+=1
    
    try:
        #Press See More Jobs button when encountered
        x=driver1.find_element(By.XPATH,"//button[@aria-label='See more jobs']")
        driver1.execute_script("arguments[0].click();",x)
        time.sleep(5)
    except:
        pass
        time.sleep(4)

## Storing the loaded HTML into a component

In [ ]:
loadedDataObj = driver1.find_element(By.TAG_NAME,"body")

In [ ]:
loadedDataHTML = loadedDataObj.get_attribute("outerHTML")

In [ ]:
loadedDataHTML

## Using BeautifulSoup to parse the html

In [ ]:
soupMain = BeautifulSoup(loadedDataHTML,"html.parser")

## Scooping the information into arrays

In [ ]:
companyNames = []
jobTitles = []
companyLocations = []
linkToJobPostings = []


In [ ]:
companyNamesHTML = soupMain.find_all("a",class_="hidden-nested-link")
jobTitlesHTML = soupMain.find_all("h3",class_="base-search-card__title")
companyLocationsHTML = soupMain.find_all("span",class_="job-search-card__location")
linkToJobPostingsHTML = soupMain.find_all("a",class_="hidden-nested-link",href=True)

In [ ]:
n = len(companyNamesHTML)
for i in range(0,n) :
    companyNames.append(companyNamesHTML[i].text)
    jobTitles.append(jobTitlesHTML[i].text)
    companyLocations.append(companyLocationsHTML[i].text)
    linkToJobPostings.append(linkToJobPostingsHTML[i]["href"])

In [ ]:
companyNames,jobTitles,companyLocations,linkToJobPostings

## Extracting more informations

In [ ]:
jobDescriptions = []
timePosted = []
relativeTime = []
jobId = []
employementDetails=[]

In [ ]:
jobListsElements = driver1.find_elements(By.XPATH,"//a[@data-tracking-control-name='public_jobs_jserp-result_search-card']")

In [ ]:
n = len(jobListsElements)
driver2 = webdriver.Chrome(executable_path=r'".\\chromedriver.exe"')
for i in range(0,n) :
    pos=""
    pageNum=""
    currentJobId=""

    currElement = jobListsElements[i].get_attribute("outerHTML")
    s2 = BeautifulSoup(currElement,"html.parser")
    link = s2.find("a",href=True)
    
    
    #calculate jobid
    t1=link["href"].find("?refId")
    s = link["href"][:t1]
    k=s.split("-")
    currentJobId=k[len(k)-1]
    
    #for pageNum
    t2 = link["href"].find("&trk")
    s2=link["href"][:t2]
    k2 = s2.split('=')
    pageNum=k2[len(k2)-1]
    
    #position
    t3=link["href"].find("&pageNum")
    s3=link["href"][:t3]
    k3=s3.split("=")
    pos=k3[len(k3)-1]
    
    lindInNewUrl = "https://www.linkedin.com/jobs/search?keywords=Business%2BIntelligence%2B%28BI%29&location=United%2BStates&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&currentJobId="+currentJobId+"&position="+pos+"&pageNum="+pageNum
    driver2.get(lindInNewUrl)
    time.sleep(5)
    
    bodyElementObj = driver2.find_element(By.TAG_NAME,"body")
    bodyElementHTML = bodyElementObj.get_attribute("outerHTML")
    soupDetails = BeautifulSoup(bodyElementHTML,"html.parser")
    
    
    
    check = soupDetails.find("h2",class_="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title")
    if check == None :
        driver2.get(lindInNewUrl)
        time.sleep(5)
    
        bodyElementObj = driver2.find_element(By.TAG_NAME,"body")
        bodyElementHTML = bodyElementObj.get_attribute("outerHTML")
        soupDetails = BeautifulSoup(bodyElementHTML,"html.parser")
    
    jobId.append(currentJobId)

    jobDescription = soupDetails.find("div",class_="show-more-less-html__markup")
    jobDescriptions.append(jobDescription)

    employementTypeList = soupDetails.find_all("span",class_="description__job-criteria-text description__job-criteria-text--criteria")
    employementDetails.append(employementTypeList)

    relativeTimeHTML1 = soupDetails.find("span",class_="posted-time-ago__text topcard__flavor--metadata")
    relativeTimeHTML2 = soupDetails.find("span",class_="posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata")
    if relativeTimeHTML1 == None :
        relativeTime.append(relativeTimeHTML2.text)
    else:
        relativeTime.append(relativeTimeHTML1.text)


In [ ]:
m=len(companyNames)

In [ ]:
jobDescriptions=jobDescriptions[:m]

In [ ]:
employementDetails=employementDetails[:m]

In [ ]:
relativeTime=relativeTime[:m]

## Creating a DataFrame

In [ ]:
df = pd.DataFrame()

In [ ]:
df['CompanyNames'] = companyNames

In [ ]:
df['JobTitles'] = jobTitles

In [ ]:
df['CompanyLocation'] = companyLocations

In [ ]:
df['LinkToJobPostings'] = linkToJobPostings

In [ ]:
df['JobDescriptions'] = jobDescriptions

In [ ]:
employementDetails[7]

In [ ]:
seniorityLevel=[]
employementType=[]
jobFunctions=[]
industries=[]

for i in range(0,len(employementDetails)):
    if len(employementDetails[i]) == 4:
        sp0 = BeautifulSoup(str(employementDetails[i][0]),"html.parser")
        sp1 = BeautifulSoup(str(employementDetails[i][1]),"html.parser")
        sp2 = BeautifulSoup(str(employementDetails[i][2]),"html.parser")
        sp3 = BeautifulSoup(str(employementDetails[i][3]),"html.parser")
        t0 = sp0.text
        t1 = sp1.text
        t2 = sp2.text
        t3 = sp3.text
        seniorityLevel.append(t0)
        employementType.append(t1)
        jobFunctions.append(t2)
        industries.append(t3)
    else :
        seniorityLevel.append("Scrapping Missed")
        employementType.append("Scrapping Missed")
        jobFunctions.append("Scrapping Missed")
        industries.append("Scrapping Missed")

In [ ]:
len(seniorityLevel)

In [ ]:
df['SeniorityLevel'] = seniorityLevel
df['EmployementType'] = employementType
df['JobFunctions'] = jobFunctions
df['Industries'] = industries

In [ ]:
df['RelativeTime'] = relativeTime

In [ ]:
df['RelativeTime']

In [ ]:
df

### Cleansing the data

In [ ]:
remchar = "\n"

In [ ]:
df['CompanyNames'] = df['CompanyNames'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [ ]:
df['JobTitles'] = df['JobTitles'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [ ]:
df['CompanyLocation'] = df['CompanyLocation'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [ ]:
df['SeniorityLevel'] = df['SeniorityLevel'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [ ]:
df['EmployementType'] = df['EmployementType'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [ ]:
df['JobFunctions'] = df['JobFunctions'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [ ]:
df['Industries'] = df['Industries'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [ ]:
df['RelativeTime'] = df['RelativeTime'].apply(lambda x : x.translate(str.maketrans('','',remchar)))

In [ ]:
len(df.iat[0,0])

In [ ]:
df['CompanyNames'] = df['CompanyNames'].apply(lambda x : x.strip())
df['JobTitles'] = df['JobTitles'].apply(lambda x : x.strip())
df['CompanyLocation'] = df['CompanyLocation'].apply(lambda x : x.strip())
df['SeniorityLevel'] = df['SeniorityLevel'].apply(lambda x : x.strip())
df['EmployementType'] = df['EmployementType'].apply(lambda x : x.strip())
df['JobFunctions'] = df['JobFunctions'].apply(lambda x : x.strip())
df['Industries'] = df['Industries'].apply(lambda x : x.strip())
df['RelativeTime'] = df['RelativeTime'].apply(lambda x : x.strip())

In [ ]:
df

In [ ]:
df['RelativeTime'][0]

## Save Data to CSV

In [ ]:
df.to_csv('scrappedData.csv', mode='a', index=False, header=False) #df.to_csv('scrappedData.csv',index=False)

## Load Data from CSV

In [ ]:
df=pd.read_csv("scrappedData.csv")

In [ ]:
df

## Push Data to SQL

In [ ]:
import pyodbc

In [ ]:
server = 'DESKTOP-50GJUM7' 
database = 'ResumeProjects' 

### Connect to SQLServer

In [ ]:
cnxn = pyodbc.connect("DRIVER={SQL Server};"
                     "Server="+server+";"+
                     "Database="+database+";"+
                      "Trusted_Connection=yes;")
cursor = cnxn.cursor()

### Create a SQL table

In [ ]:
#createTableQuery = "CREATE TABLE [LinkdInData].[JobDetails] ([CompanyNames] varchar(300),[JobTitles] varchar(500),[CompanyLocation] varchar(100),[LinkToJobPostings] varchar(800),[JobDescriptions] TEXT,[SeniorityLevel] varchar(100),[EmployementType] varchar(100),[JobFunctions] varchar(200),[Industries] varchar(200),[RelativeTime] varchar(100))"

In [ ]:
df.columns

In [ ]:
#cursor.execute(createTableQuery)
#cnxn.commit()
cursor.execute("TRUNCATE TABLE LinkdInData.JobDetails")
cnxn.commit()

In [ ]:
for index, row in df.iterrows():
     cursor.execute("INSERT INTO LinkdInData.JobDetails (CompanyNames,JobTitles,CompanyLocation,LinkToJobPostings,JobDescriptions,SeniorityLevel,EmployementType,JobFunctions,Industries,RelativeTime) values(?,?,?,?,?,?,?,?,?,?)", row.CompanyNames, row.JobTitles, row.CompanyLocation, row.LinkToJobPostings, row.JobDescriptions, row.SeniorityLevel, row.EmployementType, row.JobFunctions, row.Industries,row.RelativeTime)

In [ ]:
cnxn.commit()

In [ ]:
cursor.close()